###  Some useful imports

In [1]:
import os
import datetime 
import requests
import pandas as pd
import simplejson as json
import pydgraph

###  Generate json file with test data

> Here you want to generate a json file containing data you will be able to use to call your service  
> **You can skip / remove this step if you already have a json file with test data**  
> To use your file, simply copy it to **testdata folder** eg. testdata/european.json  
> **Data should not require any further procesing before calling the service**  

In [2]:
# retrieve apple listed options contracts from yahoo using yfinance library
# format data and save it as a json file

import os
import datetime 
import pandas as pd
import yfinance as yf

# pandas df which will contain puts and calls option quotes for all expiries
flat_df = pd.DataFrame()

# retrieve data for AAPL (Apple)
aapl = yf.Ticker("AAPL")

# main loop where we fill flat_df
for expi in aapl.options:

    # get option chain df for given expiry
    chain = aapl.option_chain(expi)
    
    # add putcall column to both calls and puts df
    chain.calls["putcall"] = "call"
    chain.puts["putcall"] = "put"

    # append both calls and puts df to flat_df and add expiry column
    flat_df = flat_df.append(chain.calls, sort=False)
    flat_df = flat_df.append(chain.puts, sort=False)
    flat_df["expiry"] = datetime.datetime.strptime(expi, '%Y-%m-%d')


# add undticker column, filter columns and rename contractSymbol column
flat_df["undticker"] = "AAPL"
flat_df = flat_df[["contractSymbol", "undticker", "expiry", "strike", "putcall"]]
flat_df = flat_df.rename(columns={'contractSymbol': 'ticker'})

# save flat_df to ./yahoo.json
res = flat_df.to_json(os.path.join(os.getcwd(),"../testdata/european.json"), orient='records', date_format='epoch')

print flat_df.head(10)
if res is not None:
    print 'an error occured'
    print res

                ticker undticker     expiry  strike putcall
0  AAPL200619C00075000      AAPL 2020-02-07    75.0    call
1  AAPL200619C00080000      AAPL 2020-02-07    80.0    call
2  AAPL200619C00085000      AAPL 2020-02-07    85.0    call
3  AAPL200619C00090000      AAPL 2020-02-07    90.0    call
4  AAPL200619C00095000      AAPL 2020-02-07    95.0    call
5  AAPL200619C00100000      AAPL 2020-02-07   100.0    call
6  AAPL200619C00105000      AAPL 2020-02-07   105.0    call
7  AAPL200619C00110000      AAPL 2020-02-07   110.0    call
8  AAPL200619C00115000      AAPL 2020-02-07   115.0    call
9  AAPL200619C00120000      AAPL 2020-02-07   120.0    call


## Call service with data from a json file

In [3]:
import simplejson as json
import requests

# load test data from json file and call service endpoint

contracts = json.load(open(os.path.join(os.getcwd(),"../testdata/european.json")))

res = [requests.post('http://localhost:3000/europeans', json=contract) for contract in contracts]
print res[:10]

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


## Check

In [ ]:
# retrieve data from service and print

res = requests.get('http://localhost:3000/europeans')
europeans = res.json()['europeans']

print res.json()
print json.dumps(europeans[:20], indent=4)
